In [1]:
import sys
sys.path.append("/Users/stevenumbarger/Research/Sarah/Jupyter_notebooks/GMC project/") 
import gizmo_analysis as gizmo 
import utilities as ut
from fof_analysis import fof
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from matplotlib import rc #to use Latex math symbols like 'phi'
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import seaborn as sns
from astropy.table import Table
from astropy.io import ascii
import pickle

In [2]:
b_parsec    = 4
b_kpc       = b_parsec / 1000.
ncut_min    = 5
age_cut_Myr = 3 #Myr
age_cut_Gyr = age_cut_Myr / 1000. #Gyr
softening   = 4. #res 7100
snaps       = [622]

simname     = 'm12m_res7100'
simdir      = '/Users/stevenumbarger/Research/Sarah/Jupyter_notebooks/GMC project/sims/m12m_res7100/'

In [3]:
part = gizmo.io.Read.read_snapshots(['star'],'index', snaps[0], simulation_directory=simdir, assign_hosts_rotation=True, assign_hosts=True)


# in utilities.simulation.Snapshot():
* reading:  Users/stevenumbarger/Research/Sarah/Jupyter_notebooks/GMC project/sims/m12m_res7100/snapshot_times.txt

  using snapshot index = 622, redshift = 0.005


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/stevenumbarger/Research/Sarah/Jupyter_notebooks/GMC project/sims/m12m_res7100/output/snapshot_622/snapshot_622.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 140876456 particles
    dark2     (id = 2): 10063684 particles
    gas       (id = 0): 116800727 particles
    star      (id = 4): 25286033 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['star']

* reading particles from:
    snapshot_622.0.hdf5
    snapshot_622.1.hdf5
    snapshot_622.2.hdf5
    snapshot_622.3.hdf5
    snapshot_622.4.hdf5
    snapshot_622.5.hdf5
    snapshot_622.6.hdf5
    snapshot_622.7.hdf5

! cannot find MUSIC config file:  Users/stevenumbarger/Research/Sarah/Jupyter_not

In [4]:
pkl_path = '/Users/stevenumbarger/Research/Sarah/Jupyter_notebooks/GMC project/Cloud_pickle_618-626'
filename = pkl_path + '/' + 'm12m_' + str(snaps[0]) + '_cluster.pkl'
with open(filename, 'rb') as handle: 
    cl_id = pickle.load(handle)

In [5]:
print(cl_id)

[[103973930, 18264393, 126053772, 31610981, 72643247, 72417720, 134341360, 131719730, 65356124, 86641923, 130181517, 94553503], [129831571, 102212918, 100842664, 102349254, 82176992, 58262478, 85532438, 73235615, 94853448, 95648522, 28580198], [51820772, 65702556, 108813177, 67530277, 124789785, 120251408, 85058328, 93965488, 46305965, 118607214], [50369798, 70186653, 83912956, 136879959, 27753485, 109624891, 69808762, 69743757, 95028052], [128220780, 117566731, 88341819, 45322848, 56950636, 93202463, 114555977, 113158390], [78885052, 33146115, 103317659, 76280924, 65466603, 124769483, 40786145, 52843196], [99392816, 68248640, 89890486, 78078696, 56107581, 80136775, 60326632, 89153868], [47178673, 71152049, 114347901, 42218516, 98204987, 74045390, 102629284, 133203755], [45052546, 132885455, 67011059, 85365756, 73344833, 20454504, 129460467], [77364754, 80053901, 132220067, 45713290, 126540086], [117478247, 69170502, 117008562, 117192278, 129867350], [65702583, 67976603, 72203838, 7937

In [6]:
print(len(cl_id))

18


In [7]:
print(cl_id[0])

[103973930, 18264393, 126053772, 31610981, 72643247, 72417720, 134341360, 131719730, 65356124, 86641923, 130181517, 94553503]


In [8]:
print(cl_id[17])

[101759372, 131930837, 56390406, 97226682, 106327377]


In [9]:
cloud_path = '/Users/stevenumbarger/Research/Sarah/Jupyter_notebooks/GMC project/Cloud_1Myr618-626/'
data = ascii.read(cloud_path + 'cloud_props_m12m_1Myr622.txt')
idata = ascii.read(cloud_path + 'cloud_indices_m12m_1Myr622.txt')

In [10]:
print(data.keys())

['cloud number', 'xcm', 'ycm', 'zcm', 'mass', 'r_90', 'vdisp_x', 'vdisp_y', 'vdisp_z', 'alpha_vir', 'alpha_vir_s']


In [11]:
print(idata.keys())

['cloud number', 'index', 'id', 'x', 'y', 'z', 'mass', 'FeH', 'OxH', 'MfFe', 'MfOx']


In [12]:
MsunToGm = 1.99e33 #conversion from mass of sun to the mass of sun in grams
KpcToCm = 3.086e21 #conversion from kiloparsecs to centimeters
mp = 1.67e-24 #mass of a proton in mega grams
#bin_edge = 10.
bin_edge = 20. #to make the grid, it will be 20 kpc from the center in all directions
##gas properties to carry around

In [13]:
#creating empty lists to fill
xg = [] #x position of gas
yg = [] #y position of gas
zg = [] #z position of gas
vxg = [] #x velocity of gas
vyg = [] #y velocity of gas
vzg = [] #z velocity of gas
mg = [] #mass of gas
rhog = [] #density of gas
tg = [] #temperature of gas
idg = [] #id number of gas

xs = [] #x position of star
ys = [] #y position of star
zs = [] #z position of star
ms = [] #mass of the star
ids = [] #id number of star
    
#star properties to carry around

sxp = [] #star x position?
syp = [] #star y position?
szp = [] #star z position?
sxp_new = [] #new star x position?
syp_new = [] #new star y position?
szp_new = [] #new star z position?
sage = [] #age of the star

gxp = [] #gas x position?
gyp = [] #gas y position?
gzp = [] #gas z position?

vdispx = [] #standard deviation of vx
vdispy = [] #standard deviation of vy
vdispz = [] #standard deviation of vz

In [14]:
print(snaps)

[622]


In [15]:
sxcm = np.zeros(shape=(len(snaps),len(cl_id))) #empty array of star center of mass in x
sycm = np.zeros(shape=(len(snaps),len(cl_id))) #empty array of star center of mass in y
szcm = np.zeros(shape=(len(snaps),len(cl_id))) #empty array of star center of mass in z
ap_num = np.zeros(shape=(len(snaps),len(cl_id))) ###Ask Sam what this is

In [16]:
r_ap = 0.05 ###Ask Sam what this is

In [17]:
point_gas = [] ###Ask Sam what this is

In [18]:
tsnap = []

In [ ]:
part['star'].prop('host.distance.principal.cylindrical')[:,0] # the star radius from the center

In [ ]:
for i in range(len(snaps)): #if i is in the range of the length of snaps
    #/home/smbeninc/scr/m12m/m12m_res7100/ #where it is located
    ################################################################################################# reads in snapshots
    #part = gizmo.io.Read.read_snapshots('all', 'snapshot_index', snaps[i],assign_host_principal_axes=True)
    #read the snapshot

    part = gizmo.io.Read.read_snapshots(['star', 'gas'],'index', snaps[i], simulation_directory=simdir, assign_hosts_rotation=True, assign_hosts=True)
    ################################################################################################# reads in snapshots
    
    tsnap.append(np.max(part['star'].prop('form.time'))) #add the max star form time to the snap times

    ig = np.where((part['gas'].prop('host.distance.principal.cylindrical')[:,0] <= bin_edge) & (np.fabs(part['gas'].prop('host.distance.principal.cartesian')[:,2]) <= 1.5) & (part['gas']['temperature'] <= 1e4))
    #get the indices where the gas cylindrical radius is less than or equal to the bin edge and 
    #the cartesian distance height is less than or equal to 1.5 and the gas temperature is less than 10000, cold
    #index gas
    
    ist = np.where((part['star'].prop('host.distance.principal.cylindrical')[:,0] <= bin_edge) & (np.fabs(part['star'].prop('host.distance.principal.cartesian')[:,2]) <= 1.5))
    #get the indices where the star cylindrical radius is less than or equal to the bin edge and
    #the cartesian distance is less than or equal to 1.5
    #can change the variable to is to be easier
    
    xg.append(part['gas'].prop('host.distance.principal.cartesian')[ig[0],0]) 
    #add the cartesian x coordinate of ig to the x values of the gas of the empty array
    yg.append(part['gas'].prop('host.distance.principal.cartesian')[ig[0],1])
    #add the cartesian y coordinate of ig to the y values of the gas
    zg.append(part['gas'].prop('host.distance.principal.cartesian')[ig[0],2])
    #add the cartesian z coordinate of ig to the z values of the gas
    
    vxg.append(part['gas'].prop('host.velocity.principal.cartesian')[ig[0],0])
    #add the x velocity of ig to the x velocity values of the gas of the empty array
    vyg.append(part['gas'].prop('host.velocity.principal.cartesian')[ig[0],1])
    #add the y velocity of ig to the y velocity values of the gas
    vzg.append(part['gas'].prop('host.velocity.principal.cartesian')[ig[0],2])
    #add the z velocity of ig to the z velocity values of the gas
    
    mg.append(part['gas']['mass'][ig])
    #add the mass of ig to the mass values of the gas
    rhog.append(part['gas'].prop('number.density')[ig])
    #add the number density of ig to the density values of the gas
    #take a look to see the density
    tg.append(part['gas']['temperature'][ig])
    #add the temperature of ig to the temperature values of the gas
    idg.append(part['gas']['id'][ig])
    #add the id of ig to the id values of the gas particle
    
    xs.append(part['star'].prop('host.distance.principal.cartesian')[ist[0],0])
    #add the cartesian x coordinate of ist to the x values of the stars
    ys.append(part['star'].prop('host.distance.principal.cartesian')[ist[0],1])
    #add the cartesian y coordinate of ist to the y values of the stars
    zs.append(part['star'].prop('host.distance.principal.cartesian')[ist[0],2])
    #add the cartesian z coordinate of ist to the z values of the stars
    ms.append(part['star']['mass'][ist])
    #add the mass of ist to the mass values of the stars
    ids.append(part['star']['id'][ist])
    #add the id of ist to the id values of the stars
    sage.append(part['star'].prop('age')[ist]*1000.) #converting age to Mega years instead of G Year
    #add the age of ist to the age values of the stars
        
    msh = np.array(ms[i])
    #assign variable msh to be the array of the mass of stars of i
    xsh = np.array(xs[i])
    #assign variable xsh to be the array of the x coordinates of the stars of i
    ysh = np.array(ys[i])
    #assign variable ysh to be the array of the y coordinates of the stars of i
    zsh = np.array(zs[i])
    #assign variable zsh to be the array of the z coordinates of the stars of i

    mgh = np.array(mg[i])
    #assign variable mgh to be the array of the mass of the gas of i
    xgh = np.array(xg[i])
    #assign variable xgh to be the array of the x coordinates of the gas of i
    ygh = np.array(yg[i])
    #assign variable ygh to be the array of the y coordinates of the gas of i
    zgh = np.array(zg[i])
    #assign variable zgh to be the array of the z coordinates of the gas of i

    vxgh = np.array(vxg[i])
    #assign variable vxgh to be the array of the x velocity of the gas of i
    vygh = np.array(vyg[i])
    #assign variable vygh to be the array of the y velocity of the gas of i
    vzgh = np.array(vzg[i])
    #assign variable vzgh to be the array of the z velocity of the gas of i
    
    for j in range(len(cl_id)):
        #when it is in the range of the length of the pickle file we opened
        cl = cl_id[j] #assign variable cl to be the points in the range of the pickle file m12m
        ind = [] #creating a list for indices that belong to a cloud
        check = [] #creating a list to look at the number of particles in each cloud
        print(j, ind, check) #print the j, ind, and check, an empty list
        for k in range(len(cl)): #making sure the actual id numbers match with id numbers
            #when it is in the range of the length of cl
            ind.append(np.where(cl[k] == ids[i])[0]) 
            #add the value of cl of k is equal to the first value of the id for stars to ind
            check.append(len(ind[k]))
            #add the length of ind of k to the check
        ind = np.array(ind)
        #set variable ind to be an array of ind
        ind2 = np.array(ind[np.where(np.array(check) == 1)[0]])
        #set variable ind2 to be an array where the array check is equal to 1 or if the check is true
        ind2 = ind2.astype(dtype='int') 
        #convert array ind2 to be a int type of array
        sxcm[i,j] = np.sum(msh[ind2]*xsh[ind2])/np.sum(msh[ind2])
        #sets up a variable as a math equation to get the star x position in centimeters
        sycm[i,j] = np.sum(msh[ind2]*ysh[ind2])/np.sum(msh[ind2])
        #sets up a variable as a math equation to get the star y position in centimeters
        szcm[i,j] = np.sum(msh[ind2]*zsh[ind2])/np.sum(msh[ind2])
        #sets up a variable as a math equation to get the star z position in centimeters
        sxp.append(xsh[ind2])
        #add the ind2 of the xsh to the star x position
        syp.append(ysh[ind2])
        #add the ind2 of the ysh to the star y position
        szp.append(zsh[ind2])
        #add the ind2 of the zsh to the star z position

        rh = np.sqrt((xgh-sxcm[i,j])**2 + (ygh-sycm[i,j])**2 + (zgh-szcm[i,j])**2)
        #calculate the radius/distance between gas and a star
        rsh = np.sqrt((xsh-sxcm[i,j])**2 + (ysh-sycm[i,j])**2 + (zsh-szcm[i,j])**2)
        #calculate the radius/distance of the stars
        ind = np.where(rh <= r_ap)
        #assigns variable ind to be the values where the density is less than r_ap
        n = np.float(len(ind[0]))
        #assigns variable n to be the floating points of the length of the first value of the array in ind
        indst = np.where((rsh <= r_ap) & (sage[i] <= np.float(i)*2.2))
        #assigns a variable where rsh is less than r_ap and the age is less than the floating points
        gxp.append(xgh[ind])
        #add the ind of the xgh to the gxp values
        gyp.append(ygh[ind])
        #add the ind of the ygh of the gyp values
        sxp_new.append(xsh[indst])
        #add the indst of the xsh to sxp_new
        syp_new.append(ysh[indst])
        #add the indst of the ysh to syp_new
        
        #appears to be a filter for "ind"
        if (len(ind[0]) > 0): #if statement if it is greater than 0
            vmean = np.average(vxgh[ind],weights=mgh[ind])
            #calculate the average and the weights
            vdispx.append(np.sqrt(np.sum(mgh[ind]*(vxgh[ind]-vmean)**2)/ ((n-1)*np.sum(mgh[ind])/n)))
            #add it to vdispx
            vmean = np.average(vygh[ind],weights=mgh[ind])
            #calculate the average and the weights
            vdispy.append(np.sqrt(np.sum(mgh[ind]*(vygh[ind]-vmean)**2)/ ((n-1)*np.sum(mgh[ind])/n)))
            #add it to vdispy
            vmean = np.average(vzgh[ind],weights=mgh[ind])
            #calculate the average and the weights
            vdispz.append(np.sqrt(np.sum(mgh[ind]*(vzgh[ind]-vmean)**2)/ ((n-1)*np.sum(mgh[ind])/n)))
            #add it to vdispz
        else: #if it is not greater than zero
            vdispx.append(0.) #add it to vdispx
            vdispy.append(0.) #add it to vdispy
            vdispz.append(0.) #add it to vdispz



# in utilities.simulation.Snapshot():
* reading:  Users/stevenumbarger/Research/Sarah/Jupyter_notebooks/GMC project/sims/m12m_res7100/snapshot_times.txt

  using snapshot index = 622, redshift = 0.005


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/stevenumbarger/Research/Sarah/Jupyter_notebooks/GMC project/sims/m12m_res7100/output/snapshot_622/snapshot_622.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 140876456 particles
    dark2     (id = 2): 10063684 particles
    gas       (id = 0): 116800727 particles
    star      (id = 4): 25286033 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['star', 'gas']

* reading particles from:
    snapshot_622.0.hdf5
    snapshot_622.1.hdf5
    snapshot_622.2.hdf5
    snapshot_622.3.hdf5
    snapshot_622.4.hdf5
    snapshot_622.5.hdf5
    snapshot_622.6.hdf5
    snapshot_622.7.hdf5

! cannot find MUSIC config file:  Users/stevenumbarger/Research/Sarah/Jupy